In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

train_df = pd.read_csv("data/updated_train.csv")

train_df.head(3)

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5


In [4]:
# adding amino acid percentage
from numpy import zeros

n = len(train_df)

# add protein sequence length column
train_df["prot_len"] = train_df.apply(lambda row: len(str(row["protein_sequence"])), axis=1)

amino_df = pd.read_csv("data/amino_acid_translation.csv")
for c in amino_df.char:
    train_df[c] = zeros(n)


def map_aa_occurences(c, row):
    return (row["protein_sequence"].count(c))/row["prot_len"]

for c in amino_df.char:
    train_df[c] = train_df.apply(lambda row: map_aa_occurences(c, row), axis=1)

train_df.head()


,seq_id,protein_sequence,pH,data_source,tm,prot_len,A,R,N,D,...,L,K,M,F,P,S,T,W,Y,V
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,0.131965,0.073314,0.014663,0.038123,...,0.108504,0.046921,0.023460,0.038123,0.052786,0.032258,0.041056,0.011730,0.008798,0.108504
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,0.097902,0.104895,0.020979,0.034965,...,0.080420,0.066434,0.006993,0.020979,0.027972,0.048951,0.041958,0.010490,0.010490,0.045455
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,0.100604,0.062374,0.030181,0.054326,...,0.036217,0.078471,0.012072,0.042254,0.040241,0.066398,0.060362,0.006036,0.032193,0.060362
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,0.075472,0.037736,0.033962,0.071698,...,0.105660,0.064151,0.007547,0.045283,0.060377,0.060377,0.071698,0.011321,0.015094,0.052830
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,0.059269,0.043418,0.044797,0.053756,...,0.071675,0.046864,0.021365,0.022054,0.088215,0.101999,0.082702,0.011027,0.032391,0.085458


In [5]:
# save list of features
from file_utils import write_json

features = [c for c in amino_df.char]
features += ["pH"]
features += ["prot_len"]

write_json("data/features.json", features)

In [7]:
# Save to disk
train_df.to_csv("data/train.csv", index=False)


In [ ]:
def check_for_nan_index():
    index = []
    for col in features:
        index_col = train_df[col].index[train_df[col].apply(np.isnan)]
        if not index_col.empty:
            print(col)
            index.append(index_col)

    return index